# Pytorch Training UI

### Import Libraries

In [3]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

In [4]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    py_file_location = "./drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch" # my private packages are stored here
    home_directory = './drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/' # my home directory is stored in ./LAB of google drive
    !pip install einops
else:
    py_file_location = './PrivatePackages/pytorch'
    home_directory = './'

sys.path.append(os.path.abspath(py_file_location))

from environment import *
from utils import *

from sklearn.model_selection import train_test_split

In [5]:
from model.model_class import LSTM, BERT, LSTM_DANN, BERT_DANN

### Set Seed and Load Data

In [6]:
SEED = 2608

In [7]:
data1 = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open(home_directory + './data/raw/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

data_test = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/test_data.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

# create domain labels for data
for i in range(len(data1)):
    data1[i]['domain'] = 0
for i in range(len(data2)):
    data2[i]['domain'] = 1

In [8]:
# Train Val Test Split

# get labels for stratification
label1 = [instance['label'] for instance in data1]
label2 = [instance['label'] for instance in data2]

train_ix_1, val_test_ix_1 = train_test_split(range(len(data1)), test_size=0.3, random_state=SEED, stratify = label1)
train_ix_2, val_test_ix_2 = train_test_split(range(len(data2)), test_size=0.3, random_state=SEED, stratify = label2)
val_ix_1, test_ix_1 = train_test_split(val_test_ix_1, test_size=0.5, random_state=SEED, stratify = [data1[i]['label'] for i in val_test_ix_1])
val_ix_2, test_ix_2 = train_test_split(val_test_ix_2, test_size=0.5, random_state=SEED, stratify = [data2[i]['label'] for i in val_test_ix_2])

# split data according to the index from train_test_split
train_data_1 = [data1[i] for i in train_ix_1]
val_data_1 = [data1[i] for i in val_ix_1]
test_data_1 = [data1[i] for i in test_ix_1]
train_data_2 = [data2[i] for i in train_ix_2]
val_data_2 = [data2[i] for i in val_ix_2]
test_data_2 = [data2[i] for i in test_ix_2]

# combine the data
train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2
test_data = test_data_1 + test_data_2

---
Preprocess data

In [9]:
def W2V_DataFactory(data: list, context_window: int, seed: int, raw_token_pytorch_map: dict, k) -> list:

    """ Get W2V training data """
    
    assert context_window % 2 == 1, 'context window must be odd'

    np.random.seed(seed)

    MAX_SAMPLED_NEGATIVE_TOKENS = 10000

    retokenised_keys = list(raw_token_pytorch_map.keys())

    negative_tokens = np.random.choice(retokenised_keys, MAX_SAMPLED_NEGATIVE_TOKENS)

    negative_up_to = 0

    w2v_data = []

    for instance in tqdm(data):
        tokens = [context_window//2 * 'CLS'] + instance['text'] + [context_window//2 * raw_token_pytorch_map['PAD']]

        for i in range(context_window//2, len(tokens) - context_window//2):
            
            focus_token_retokenised = raw_token_pytorch_map.get(tokens[i], raw_token_pytorch_map['UNK'])
            context_words = set()

            for j in range(-context_window//2, context_window//2+1):
                if j != 0: # don't want to make positive sample with self
                    if tokens[j] in context_words: # CLS and Padding (being start and end) being repeated
                        continue 
                    
                    new_instance = {'token': focus_token_retokenised, 'context': raw_token_pytorch_map.get(tokens[j], raw_token_pytorch_map['UNK']), 'label': 1}
                    w2v_data.append(new_instance)
                    context_words.add(tokens[j])
            
            for j in range(len(context_words)): # sample the same number of negatives
                # TODO: different for each round?
                while True:
                    
                    if negative_up_to == MAX_SAMPLED_NEGATIVE_TOKENS:
                        negative_up_to = 0
                        #TODO: shuffle

                    sampled_negative_retokenised = negative_tokens[negative_up_to]
                    negative_up_to += 1
                    if sampled_negative_retokenised not in context_words: # didn't sample a positive case
                        break

                new_instance = {'token': focus_token_retokenised, 'context': sampled_negative_retokenised, 'label': 0}
                w2v_data.append(new_instance)
    
    return w2v_data

In [10]:
def BERT_pretrain_Generation(data: list, seed: int, raw_token_pytorch_map: dict, MAX_SENTENCE_LENGTH):
    
    np.random.seed(seed)

    MAX_SAMPLED_PROBS = 10000

    mask_randomness = np.random.uniform(0, 1, size=MAX_SAMPLED_PROBS)

    negative_up_to = 0

    bert_data = []

    for instance in tqdm(data):
        

        tokens = instance['text']

        tokens = [raw_token_pytorch_map['CLS']] + [raw_token_pytorch_map.get(token, raw_token_pytorch_map['UNK']) for token in tokens]
        tokens = tokens[:MAX_SENTENCE_LENGTH]

        # 15% of tokens are random
        masked_token_positions = np.random.choice(range(len(tokens)), int(0.15 * len(tokens)), False)
        
        # 80% becomes [MASK], 15% becomes random, 10% unchanged ith token, 10% random token
        for masked_token_position in masked_token_positions:
            if mask_randomness[negative_up_to] < 0.8:
                tokens[masked_token_position] = raw_token_pytorch_map['MASK']
            elif mask_randomness[negative_up_to] > 0.9:
                tokens[masked_token_position] = np.random.choice(tokens)
            
        
        tokens = tokens + [raw_token_pytorch_map['PAD']] * (MAX_SENTENCE_LENGTH - len(tokens))

        for masked_token_position in masked_token_positions:
            new_instance = {}
            new_instance['input'] = tokens
            new_instance['label'] = tokens[masked_token_position]
            new_instance['mask'] = masked_token_position
            new_instance['domain'] = instance['domain']
            bert_data.append(new_instance)
    
    return bert_data


In [11]:
def BERT_pretrain_DataFactory(train_data, val_data, seed, raw_token_pytorch_map, MAX_SENTENCE_LENGTH):
        
    train_data = BERT_pretrain_Generation(train_data, seed, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)
    val_data = BERT_pretrain_Generation(val_data, seed, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

    train_x = [instance['input'] for instance in train_data]
    train_y = [instance['label'] for instance in train_data]
    train_mask = [instance['mask'] for instance in train_data]
    train_domain = [instance['domain'] for instance in train_data]
    val_x = [instance['input'] for instance in val_data]
    val_y = [instance['label'] for instance in val_data]
    val_mask = [instance['mask'] for instance in val_data]
    val_domain = [instance['domain'] for instance in val_data]
    

    return train_x, train_y, train_mask, train_domain, val_x, val_y, val_mask, val_domain

In [16]:
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

100%|██████████| 2700/2700 [00:00<00:00, 4138.13it/s]


In [12]:
MAX_SENTENCE_LENGTH = 512
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right

In [14]:
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)

100%|██████████| 4000/4000 [00:00<00:00, 102430.01it/s]


In [15]:
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY) 

100%|██████████| 12600/12600 [00:00<00:00, 31785.90it/s]


In [19]:
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=True, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)

100%|██████████| 4000/4000 [00:00<00:00, 72797.24it/s]


In [12]:
train_x = train_x[:200]
train_y = train_y[:200]
train_dom = train_dom[:200]
val_x = val_x[:200]
val_y = val_y[:200]
val_dom = val_dom[:200]

In [13]:
pos_prior, neg_prior = get_distribution(train_y)
pos_prior, neg_prior

(0.53, 0.47)

In [14]:
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
pos_dom_prior, neg_dom_prior

(0.0, 1.0)

In [ ]:
# class Dataset():
#     """ Pytorch style dataset """

#     def __init__(self, data, maxlen):
#         self.data = data
#         self.maxlen = maxlen
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         return self.data[index]['text'], self.data[index]['label']
#         # return self.data[index]['text'], self.data[index]['label'], self.data[index]['domain']

---
### Models

#### 1. Prediction only

In [ ]:
# LSTM

class LSTM_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 1
    bidirectional = True
    n_heads = 8
    dropout = 0.1
    n_mlp_layers = 0
    flatten = False
    activation = nn.ReLU()
    res_learning = False
    mask_flag = False # TODO
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 5
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    regularisation_loss = None
    scheduler = True
    grad_clip = True
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_Classifier'
    


model = LSTM(LSTM_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/2 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [16]:
# BERT

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 128
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3 
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 0
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 2
    lr = 1e-3
    patience = 2
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'
    


model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 2/2 [00:03<00:00,  1.81s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali

Epoch 1 Train | Loss:  0.7894 | Accuracy:  0.4900| F1:  0.5641 | Balanced Accuracy:  0.4815 |  
                    Domain 1 Accuracy:  0.4900| Domain 1 F1:  0.5641 | Domain 1 Balanced Accuracy:  0.4815 | 
                    Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

Epoch 1 Val   | Loss:  0.5774 | Accuracy:  0.5100| F1:  0.0000 | Balanced Accuracy:  0.5000 | 
                Domain 1 Accuracy:  0.5100| Domain 1 F1:  0.0000 | Domain 1 Balanced Accuracy:  0.5000 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


100%|██████████| 2/2 [00:03<00:00,  1.60s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-pac

Epoch 2 Train | Loss:  0.4621 | Accuracy:  0.5200| F1:  0.4000 | Balanced Accuracy:  0.5339 |  
                    Domain 1 Accuracy:  0.5200| Domain 1 F1:  0.4000 | Domain 1 Balanced Accuracy:  0.5339 | 
                    Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

Epoch 2 Val   | Loss:  0.5292 | Accuracy:  0.4900| F1:  0.6577 | Balanced Accuracy:  0.5000 | 
                Domain 1 Accuracy:  0.4900| Domain 1 F1:  0.6577 | Domain 1 Balanced Accuracy:  0.5000 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan

Epoch 2 Val   | Loss:  0.5292 | Accuracy:  0.4900| F1:  0.6577 | Balanced Accuracy:  0.5000 | 
                Domain 1 Accuracy:  0.4900| Domain 1 F1:  0.6577 | Domain 1 Balanced Accuracy:  0.5000 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

---
# DANN

In [ ]:
# LSTM_DANN

class LSTM_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 1
    bidirectional = True
    n_heads = 0
    dropout = 0.1
    n_mlp_clf_layers = 0
    n_mlp_dom_layers = 0
    flatten = False
    activation = nn.ReLU()
    res_learning = False
    mask_flag = False # TODO
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 5
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    alpha = 0.5
    regularisation_loss = None
    scheduler = True
    grad_clip = True
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_DANN_Classifier'
    


model = LSTM_DANN(LSTM_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|█████████

Epoch 1 Train | Classification Loss:  0.3456 | Accuracy:  0.4900| F1:  0.6047 | Balanced Accuracy:  0.4743 | Domain Loss:  0.3872 | Domain Accuracy:  0.4900


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1 Val   | Classification Loss:  0.3059 | Accuracy:  0.9400| F1:  0.9691 | Balanced Accuracy:  0.9400 | Domain Loss:  1.0035 | Domain Accuracy:  0.0600


  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


...

  0%|          | 0/2 [00:00<?, ?it/s]/Users/tg.chenny/Desktop/1. University/2. Masters/7. Statistical Machine Learning/Assignments/A1/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch/model/model_class/__template__.py:292: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(), 2)
100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


Epoch 7 Train | Classification Loss:  0.3306 | Accuracy:  0.6600| F1:  0.7500 | Balanced Accuracy:  0.6407 | Domain Loss:  0.2772 | Domain Accuracy:  0.6600


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 7 Val   | Classification Loss:  0.3336 | Accuracy:  0.7350| F1:  0.8473 | Balanced Accuracy:  0.7350 | Domain Loss:  0.4540 | Domain Accuracy:  1.0000

Epoch 5 Val   | Classification Loss:  0.3042 | Accuracy:  0.9450| F1:  0.9717 | Balanced Accuracy:  0.9450 | Domain Loss:  0.7634 | Domain Accuracy:  0.0250


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [17]:
# BERT

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 128
    d_ff = 512 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 3 
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 0
    n_mlp_dom_layers = 0
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = False
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-3
    patience = 32
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    alpha = 0.5
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'
    


model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)

100%|██████████| 2/2 [00:02<00:00,  1.37s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-pac

Epoch 1 Train | Classification Loss:  0.3791 | Accuracy:  0.5100| F1:  0.4494 | Balanced Accuracy:  0.5185 | 
                Domain Loss:  1.3177 | Domain Accuracy:  0.0000 | 
                Domain 1 Accuracy:  0.5100| Domain 1 F1:  0.4494 | Domain 1 Balanced Accuracy:  0.5185 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

Epoch 1 Val   | Classification Loss:  0.3434 | Accuracy:  0.5550| F1:  0.5616 | Balanced Accuracy:  0.5555 |  
                            Domain Loss:  6.8106 | Domain Accuracy:  0.0000 |  
                            Domain 1 Accuracy:  0.5550| Domain 1 F1:  0.5616 | Domain 1 Balanced Accuracy:  0.5555 |  
                            Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


100%|██████████| 2/2 [00:03<00:00,  1.62s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-pac

...

/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

Epoch 30 Val   | Classification Loss:  0.3445 | Accuracy:  0.5550| F1:  0.5083 | Balanced Accuracy:  0.5533 |  
                            Domain Loss:  1.3859 | Domain Accuracy:  0.0000 |  
                            Domain 1 Accuracy:  0.5550| Domain 1 F1:  0.5083 | Domain 1 Balanced Accuracy:  0.5533 |  
                            Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


100%|██████████| 2/2 [00:03<00:00,  1.54s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-pac

Epoch 31 Train | Classification Loss:  0.3133 | Accuracy:  0.6750| F1:  0.6701 | Balanced Accuracy:  0.6783 | 
                Domain Loss:  0.6697 | Domain Accuracy:  0.0000 | 
                Domain 1 Accuracy:  0.6750| Domain 1 F1:  0.6701 | Domain 1 Balanced Accuracy:  0.6783 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


KeyboardInterrupt: 

---
# Evaluation and Inference

In [ ]:
model.load()
model.eval(val_x, val_y, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, best_epoch, evaluation_mode = True)

In [ ]:
EXPERIMENT_NAME = '2lstm_unidir_512d_8_512t40_wbce'

In [ ]:
future_pred_y = model.predict(future_x)

future_pred_y = [1 if x[1] > x[0] else 0 for x in future_pred_y]

predictions = pd.DataFrame({'id': range(len(future_pred_y)), 'class': future_pred_y})
predictions.to_csv(home_directory + f'predictions/{EXPERIMENT_NAME}_classification.csv', index=False)